In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import struct
import csv
import os
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [ ]:
#this function reads a particular Casimir output
def read_casimir(file,mode = 1):
    with open(file,'rb') as f:
        content = f.read()
    idx_start = content.index(str.encode('\n'))
    length = int(len(content[idx_start+1:])/8)
    num = []
    for i in range(length):
        num.append(struct.unpack("d", content[8*i+idx_start+1:8*i+8+idx_start+1])[0])

    time = float(content[content.index(str.encode('='))+2:content.index(str.encode('\n'))])


    str_chis = str(content[content.index(str.encode(':'))+1:content.index(str.encode('Cas'))])[3:-1]

    chis = [float(x) for x in str_chis.split(',')]

    num_chis = len(chis)
    if mode > 0:
        num_meshblocks = int(len(num)/(6+2+num_chis))
    else:
        num_meshblocks = int(len(num)/(6+1+num_chis))

    chunk_size = int(len(num)/num_meshblocks)
    x1_sizes = []
    x2_sizes = []
    x3_sizes = []
    for i in range(num_meshblocks):
        x1_sizes.append((num[i*chunk_size],num[i*chunk_size+1]))
        x2_sizes.append((num[i*chunk_size+2],num[i*chunk_size+3]))
        x3_sizes.append((num[i*chunk_size+4],num[i*chunk_size+5]))

    cas_momentas_list = []
    nparticle_list = []
    for i in range(num_meshblocks):
        if mode > 0:
            nparticle_list.append(num[i*chunk_size+7])
            cas_momentas = num[i*chunk_size+8:i*chunk_size+chunk_size]
            cas_momentas_list.append(cas_momentas)
        else:
            nparticle_list.append(num[i*chunk_size+6])
            cas_momentas = num[i*chunk_size+7:i*chunk_size+chunk_size]
            cas_momentas_list.append(cas_momentas)

    return chis,time,x1_sizes,x2_sizes,x3_sizes,nparticle_list,cas_momentas_list

In [ ]:
#this reads all the casimir outputs at once (once supplied with the output folder)
#sometimes reading every casimir output is more than needed, so the modular parameter can filter and select every modular output to process
#keep mode = 1 (mode = 0 is for an older version)
def read_all_casimirs(output_folder,modular=1,mode =1):
    directory = output_folder+'/mpiio/cas/'
    times = []
    cas_momentas_list_list = []
    nparticle_list_list = []
    chis = None
    for name in os.listdir(directory):
        if name != '.DS_Store' and name != '.ipynb_checkpoints':
            chis,time,x1_sizes,x2_sizes,x3_sizes,nparticle_list,cas_momentas_list = read_casimir(os.path.join(directory,name),mode=mode)
            if int(name[9:-4]) % modular == 0:
                times.append(time)
                cas_momentas_list_list.append(cas_momentas_list)
                nparticle_list_list.append(nparticle_list)

    return chis,times,x1_sizes,x2_sizes,x3_sizes,nparticle_list_list,cas_momentas_list_list

In [ ]:
def nbf(filename):

  # read the binary file
  f = open(filename,'rb')
  line = f.readline()

  # check if the file was written correctly
  if (line.split()[0] != b"Pegasus++"):
    print("Error in nbf(): unexpected file format")
    return

  # read output information
  time = np.float64(line.split()[-1]) # line 1 - output time
  line = f.readline()
  # line 2 - endianess (not used for now, but may be useful of certain machines)
  line = f.readline()
  nbtotal = int(line.split()[-1])     # line 3 - number of meshblocks
  line = f.readline()
  nvars =   int(line.split()[-1])     # line 4 - number of variables
  line = f.readline()
  var_list = line.split()[1:nvars+1]  # line 5 - list of variables
  line = f.readline()

  # lines 6-8 - read mesh information
  temp_split = line.split()
  Nx1 = int(temp_split[1].decode('ascii').split('=')[1])
  X1min = np.float64(temp_split[2].decode('ascii').split('=')[1])
  X1max = np.float64(temp_split[3].decode('ascii').split('=')[1])
  line = f.readline()
  temp_split = line.split()
  Nx2 = int(temp_split[0].decode('ascii').split('=')[1])
  X2min = np.float64(temp_split[1].decode('ascii').split('=')[1])
  X2max = np.float64(temp_split[2].decode('ascii').split('=')[1])
  line = f.readline()
  temp_split = line.split()
  Nx3 = int(temp_split[0].decode('ascii').split('=')[1])
  X3min = np.float64(temp_split[1].decode('ascii').split('=')[1])
  X3max = np.float64(temp_split[2].decode('ascii').split('=')[1])
  line = f.readline()

  # line 9 - meshblock size
  temp_split = line.split()
  nx1 = int(temp_split[1].decode('ascii').split('=')[1])
  nx2 = int(temp_split[2].decode('ascii').split('=')[1])
  nx3 = int(temp_split[3].decode('ascii').split('=')[1])

  # final result = dictionary of arrays
  result = {}
  # add an entry for each variable
  for nv in range(nvars):
    result[var_list[nv]] = np.zeros((Nx3,Nx2,Nx1))

  # starting indices for each logical location
  islist = np.arange(0,Nx1,nx1)
  jslist = np.arange(0,Nx2,nx2)
  kslist = np.arange(0,Nx3,nx3)

  # loop over all meshblocks and read all variables
  for nb in range(nbtotal):
    il1 = struct.unpack("@i",f.read(4))[0]
    il2 = struct.unpack("@i",f.read(4))[0]
    il3 = struct.unpack("@i",f.read(4))[0]
    mx1 = struct.unpack("@i",f.read(4))[0]
    minx1 = struct.unpack("@f",f.read(4))[0]
    maxx1 = struct.unpack("@f",f.read(4))[0]
    mx2 = struct.unpack("@i",f.read(4))[0]
    minx2 = struct.unpack("@f",f.read(4))[0]
    maxx2 = struct.unpack("@f",f.read(4))[0]
    mx3 = struct.unpack("@i",f.read(4))[0]
    minx3 = struct.unpack("@f",f.read(4))[0]
    maxx3 = struct.unpack("@f",f.read(4))[0]
    iis = islist[il1]
    iie = iis + mx1
    ijs = jslist[il2]
    ije = ijs + mx2
    iks = kslist[il3]
    ike = iks + mx3
    fmt = "@%df"%(mx1*mx2*mx3)
    for nv in range(nvars):
      tmp = result[var_list[nv]]
      data = np.array(struct.unpack(fmt,f.read(4*mx1*mx2*mx3)))
      data = data.reshape(mx3,mx2,mx1)
      tmp[iks:ike,ijs:ije,iis:iie] = data
  # close the file
  f.close()

  return time,result

In [ ]:
#this reads all the nbf outputs (given the output folder) EXCEPT for the ptensors
def all_nbfs(output_folder):
    directory = output_folder+'/mpiio/grid/'
    results = []
    times = []
    for name in os.listdir(directory):
        time,result = nbf(os.path.join(directory,name))
        if not(b'pressure_tensor_11' in result.keys()):
            results.append(result)
            times.append(time)
    return times, results

In [ ]:
#this reads all the ptensor nbf outputs
def all_ptensors(output_folder):
    directory = output_folder+'/mpiio/grid/'
    results = []
    times = []
    for name in os.listdir(directory):
        time,result = nbf(os.path.join(directory,name))
        if b'pressure_tensor_11' in result.keys():
            results.append(result)
            times.append(time)
    return times, results

In [ ]:
#this function essentially does a 'weighted average' of the local casimir momenta (which is in the output) to produce an overall casimir momenta
def mesh_cas_momenta_from_mb(chi,nparticle_mb_list,cas_momentum_mb_list,mb_vol,dim=1):
    total_N = np.sum(nparticle_mb_list)
    total_V = len(nparticle_mb_list)*mb_vol
    if chi != 1:
        sum_thing = 0
        for i,nparticle in enumerate(nparticle_mb_list):
            sum_thing += nparticle * ( ( cas_momentum_mb_list[i] * ((nparticle/mb_vol)**(-1/dim)) ) ** (-(dim*chi-dim)) )
        cas_momentum_m_avg = ((total_N/total_V)**(1/dim)) * ((sum_thing/total_N) ** (-1/(dim*chi-dim)))
    elif chi == 1:
        cas_momentum_m_avg = ((total_N/total_V)**(1/dim))
        for i,nparticle in enumerate(nparticle_mb_list):
            cas_momentum_m_avg = cas_momentum_m_avg * ((mb_vol/nparticle)**(nparticle/(dim*total_N))) * (cas_momentum_mb_list[i])**(nparticle/total_N)

    return cas_momentum_m_avg

In [ ]:
#reads a distribution function (or EdotV) output, specalised for 1D
def read_1dspectrum(filepath,numsplit=1,nprl=300):
    with open(filepath, 'rb') as f:
        time_line = f.readline()
        content = f.read()
    time = np.float64(time_line.split()[-1]) # line 1 - output time
    length = int(len(content)/8)
    num = []
    for i in range(length):
        num.append(struct.unpack("d", content[8*i:8*i+8])[0])


    chunk_size = numsplit*nprl+6
    num_meshblocks = int(len(num)/chunk_size)

    x1_sizes = []
    x2_sizes = []
    x3_sizes = []
    spectrums = []
    for i in range(num_meshblocks):
        x1_sizes.append((num[i*chunk_size],num[i*chunk_size+1]))
        x2_sizes.append((num[i*chunk_size+2],num[i*chunk_size+3]))
        x3_sizes.append((num[i*chunk_size+4],num[i*chunk_size+5]))
        spectrum = num[i*chunk_size+6:(i+1)*chunk_size]
        spectrum = np.reshape(spectrum,(numsplit,nprl),order='C')
        spectrums.append(spectrum)

    return time, x1_sizes,x2_sizes,x3_sizes, spectrums

In [ ]:
#reads all output distribution functions (need numsplit and nprl from the input file)
def read_all_spectra(output_folder,numsplit=1,nprl=300):
    directory = output_folder+'/mpiio/spec/'
    times = []
    spectrums_s = []
    for name in os.listdir(directory):
        if '.spec' in name:
            time, x1_sizes,x2_sizes,x3_sizes,spectrums = read_1dspectrum(os.path.join(directory,name),numsplit,nprl)
            times.append(time)
            spectrums_s.append(spectrums)
    return times, x1_sizes, x2_sizes, x3_sizes, spectrums_s

In [ ]:
#reads all output edotvprl (need numsplit and nprl from the input file) --- same structure as distribution function
def read_all_edotvprl(output_folder,numsplit,nprl):
    directory = output_folder+'/mpiio/spec/'
    times = []
    spectrums_s = []
    for name in os.listdir(directory):
        if '.edotv_prl' in name:
            time, x1_sizes,x2_sizes,x3_sizes,spectrums = read_1dspectrum(os.path.join(directory,name),numsplit,nprl)
            times.append(time)
            spectrums_s.append(spectrums)
    return times, x1_sizes, x2_sizes, x3_sizes, spectrums_s

In [ ]:
#user-provided output directory
output_directory = 'iaw_turbulence/dedt2_tetp1_tcorr16/output'

In [ ]:
#reads the Casimir values, and also calculates the holistic Casimir momenta from the local Casimir momentas

modular = 10 #read in every 10th casimir output
dim = 1
chis,times,x1_sizes,x2_sizes,x3_sizes,nparticle_list_list,cas_momentas_list_list = read_all_casimirs(output_directory,modular,mode=1)
times, cas_momentas_list_list,nparticle_list_list = zip(*sorted(zip(times, cas_momentas_list_list,nparticle_list_list)))

cas_momentas_mesh = np.zeros((len(times),len(chis)))
mb_vol = (x1_sizes[0][1]-x1_sizes[0][0])

for itime in range(len(times)):
    for ichi,chi in enumerate(chis):
        cas_momentas_mesh[itime,ichi] = mesh_cas_momenta_from_mb(chi,nparticle_list_list[itime],np.array(cas_momentas_list_list)[itime,:,ichi],mb_vol,dim=dim)


In [ ]:
#reads the nbf and ptensor outputs (contains info about electric field, pressure etc)

times_nbf,results_nbf = all_nbfs(output_directory)
times_nbf,results_nbf = zip(*sorted(zip(times_nbf, results_nbf)))
times_p_nbf,ptensor_nbf = all_ptensors(output_directory)
times_p_nbf,ptensor_nbf = zip(*sorted(zip(times_p_nbf, ptensor_nbf)))

In [ ]:
#reads in the distribution functions, and also reshapes and reorders
#also defines the meshgrid for X and V

nprl = 600
numsplit = 10
x_mesh_max = 16.0
v_max = 4.0

times_s, x1_sizes_s, x2_sizes_s, x3_sizes_s, spectrums_s = read_all_spectra(output_directory,nprl=nprl,numsplit=numsplit)

spectrums_s = np.array(spectrums_s)
spectrums_s_reshaped = np.reshape(spectrums_s,(spectrums_s.shape[0],spectrums_s.shape[1]*spectrums_s.shape[2],spectrums_s.shape[3]))


Xs,Vs = np.meshgrid( np.linspace(0,x_mesh_max,spectrums_s_reshaped.shape[1]), np.linspace(-v_max,v_max,spectrums_s.shape[3]) )


times_s_ordered, spectrums_s_ordered = zip(*sorted(zip(times_s, spectrums_s_reshaped),key=lambda x: x[0]))


In [ ]:
#reads in the Edotvprl, and also reshapes and reorders

times_e, x1_sizes_e, x2_sizes_e, x3_sizes_e, spectrums_e = read_all_edotvprl(output_directory,nprl=nprl,numsplit=numsplit)

spectrums_e = np.array(spectrums_e)
spectrums_e_reshaped = np.reshape(spectrums_e,(spectrums_e.shape[0],spectrums_e.shape[1]*spectrums_e.shape[2],spectrums_e.shape[3]))

Xs_e,Vs_e = np.meshgrid( np.linspace(0,x_mesh_max,spectrums_e_reshaped.shape[1]), np.linspace(-v_max,v_max,spectrums_e.shape[3]) )


times_e_ordered, spectrums_e_ordered = zip(*sorted(zip(times_e, spectrums_e_reshaped),key=lambda x: x[0]))


In [ ]:
#for each chi value, plots the holistic Casimir momenta over time

fig,axs = plt.subplots(3,3,figsize=(15,10))

for i in range(9):
    axs[int(i/3),i%3].set_title(r'$\chi=$'+str(chis[i]))
    axs[int(i/3),i%3].scatter(np.array(times),cas_momentas_mesh[:,i],marker='.')

    if i ==1:
        axs[int(i/3),i%3].legend()

    axs[int(i/3),i%3].set_xlabel(r'$t$')
    axs[int(i/3),i%3].set_ylabel(r'$p^{\text{mesh}}_{c,\chi}$')


fig.tight_layout()


In [ ]:
#produces a video of the local Casimir momenta for each chi across real space. Also plots as a dotted black line the density.

data = np.array(cas_momentas_list_list)
dens_data = np.array(nparticle_list_list)

vmin = np.min(data)
vmax = np.max(data)

dens_vmin = np.min(dens_data)
dens_vmax = np.max(dens_data)

fig, ax = plt.subplots()

lines = ax.plot(np.linspace(0,x_mesh_max,len(data[0])),data[0])
ax.set_title(r'$t=$'+str(times[0]))
ax.set_ylim(vmin,vmax)
ax.set_ylabel(r'$p_{c,\chi}$')
ax.set_xlabel(r'$x$')

ax2 = ax.twinx()
dens_line = ax2.plot(np.linspace(0,x_mesh_max,len(nparticle_list_list[0])),nparticle_list_list[0],'k--')[0]
ax2.set_ylim(dens_vmin,dens_vmax)
ax2.set_ylabel(r'$n$')

def update(frame):
    for coll in ax.collections:
        coll.remove()
    for iline,line in enumerate(lines):
        line.set_ydata(data[frame,:,iline])
    dens_line.set_ydata(nparticle_list_list[frame])
    ax.set_title(r'$t=$'+str(times[frame]))
    return (line)

ani = FuncAnimation(fig, update, frames=len(data))

HTML(ani.to_jshtml())

In [ ]:
#produces a video of the distribution function in x and v

contour_data = np.array(spectrums_s_ordered)

vmin = np.min(contour_data)
vmax = np.max(contour_data)



fig, ax = plt.subplots()
from mpl_toolkits.axes_grid1 import make_axes_locatable
div = make_axes_locatable(ax)
cax = div.append_axes('right', '5%', '5%')
contour = ax.pcolormesh(Xs,Vs,contour_data[0].T,vmin=vmin,vmax=vmax)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$v$')

ax.set_title(r'$t=$'+str(times_s_ordered[0]))
cbar = fig.colorbar(contour, cax=cax)
def update(frame):
    for coll in ax.collections:
        coll.remove()
    contour = ax.pcolormesh(Xs,Vs,contour_data[frame].T,vmin=vmin,vmax=vmax)

    cax.cla()
    ax.set_title(r'$t=$'+str(times_s_ordered[frame]))
    cbar = fig.colorbar(contour, cax=cax)

ani = FuncAnimation(fig, update, frames=len(contour_data), blit=False)

HTML(ani.to_jshtml())

In [ ]:
#produces a video of the edotvprl in x and v

contour_data = np.array(spectrums_e_ordered)

vmin = np.min(contour_data)
vmax = np.max(contour_data)



fig, ax = plt.subplots()
from mpl_toolkits.axes_grid1 import make_axes_locatable
div = make_axes_locatable(ax)
cax = div.append_axes('right', '5%', '5%')
contour = ax.pcolormesh(Xs_e,Vs_e,contour_data[0].T,vmin=vmin,vmax=vmax)
ax.set_title(r'$t=$'+str(times_e_ordered[0]))
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$v$')
cbar = fig.colorbar(contour, cax=cax)
def update(frame):
    for coll in ax.collections:
        coll.remove()
    contour = ax.pcolormesh(Xs_e,Vs_e,contour_data[frame].T,vmin=vmin,vmax=vmax)

    cax.cla()
    ax.set_title(r'$t=$'+str(times_e_ordered[frame]))
    cbar = fig.colorbar(contour, cax=cax)

ani = FuncAnimation(fig, update, frames=len(contour_data), blit=False)

HTML(ani.to_jshtml())

In [ ]:
#calculates the spectrum of the electric field energy
Ecc1 = np.array([x[b'Ecc1'] for x in results_nbf]).squeeze()

E_fft = np.fft.rfft(Ecc1,axis=1)
E_fft_2 = np.abs(E_fft)**2

ks = np.fft.rfftfreq(Ecc1.shape[1])*4*np.pi/(16.0/1920)

In [ ]:
#plots a video of the electric field spectrum

fig, ax = plt.subplots()

vmin = np.min(E_fft_2)
vmax = np.max(E_fft_2)


line = ax.loglog(ks,E_fft_2[0])[0]
ax.set_title(r'$t=$'+str(times_nbf[0]))
ax.set_xlabel(r'$k$')
ax.set_ylabel(r'$|E_k|^2$')
ax.set_ylim(vmin,vmax)
ax.loglog(ks,ks)
ax.loglog(ks,np.ones(np.shape(ks))*1e4)

def update(frame):
    for coll in ax.collections:
        coll.remove()
    line.set_ydata(E_fft_2[frame])
    ax.set_title(r'$t=$'+str(times_nbf[frame]))
    return (line)

ani = FuncAnimation(fig, update, frames=len(contour_data))

HTML(ani.to_jshtml())

In [ ]:
#calculates the spectrum for the distribution function squared in position and velocity

spectrum_ffts_2 = []
for itime in range(len(times_s_ordered)):
    spectrum = np.array(spectrums_s_ordered)[itime].squeeze()
    spectrum_fft = np.fft.fftshift(np.fft.fft2(spectrum))
    spectrum_ffts_2.append(np.abs(spectrum_fft)**2)
spectrum_ffts_2 = np.array(spectrum_ffts_2)

dl_df = 16.0/(96*numsplit*80) #length divided by spacing of distribution function in real space
dv_df = 8.0/600

ks = np.fft.fftshift(np.fft.fftfreq(spectrum_ffts_2[0].shape[0],d=dl_df))
ss = np.fft.fftshift(np.fft.fftfreq(spectrum_ffts_2[0].shape[1],d=dv_df))

K,S = np.meshgrid(ks,ss)

In [ ]:
#plots a video of the spectrum of the distribution function squared

contour_data = np.array(spectrum_ffts_2)

vmin = np.min(np.log(contour_data))
vmax = np.max(np.log(contour_data))



fig, ax = plt.subplots()
from mpl_toolkits.axes_grid1 import make_axes_locatable
div = make_axes_locatable(ax)
cax = div.append_axes('right', '5%', '5%')
contour = ax.pcolormesh(K,S,np.log(contour_data[0]).T,vmin=vmin,vmax=vmax)
ax.set_title(r'$t=$'+str(times_s_ordered[0]))


ax.set_xlabel(r'$k$')
ax.set_ylabel(r'$s$')
cbar = fig.colorbar(contour, cax=cax)
# Function to update the contour plot for each frame
def update(frame):
    # ax.clear()  # Clear the previous plot
    for coll in ax.collections:
        coll.remove()
    contour = ax.pcolormesh(K,S,np.log(contour_data[frame]).T,vmin=vmin,vmax=vmax)

    cax.cla()
    ax.set_title(r'$t=$'+str(times_s_ordered[frame]))

    cbar = fig.colorbar(contour, cax=cax)

# Create the animation
ani = FuncAnimation(fig, update, frames=len(contour_data), blit=False)

# Display the animation in the Jupyter Notebook
HTML(ani.to_jshtml())